In [1]:
from keras.layers import Dense, Dropout, Activation, Flatten, Convolution2D, MaxPooling2D, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from keras.models import Sequential
from typing import Tuple, List
import tensorflow as tf
from tensorflow.random import set_seed
from tqdm import tqdm
import pandas as pd
import numpy as np
import sklearn
import keras
import os

In [2]:
#Seeding random state to 13 always, for reproducibility
np.random.seed(13)
set_seed(13)

In [3]:
#Helper Function: Return the paths to all jpg files found within a directory
def getImageDirs(root: str = "data"):
    imageDirs = []
    for subDirectory, directory, files in os.walk(root):
        for file in files:
            if file[-4:] == ".jpg":
                path = os.path.join(subDirectory, file)         
                imageDirs.append(path)
    return(imageDirs)

In [4]:
#Helper Function: Return the class weights given a list of classes
def getClassWeightsFromLabels(labels: List[int]):# -> Dict[int]:
    weights = sklearn.utils.class_weight.compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels)
    return {0: weights[0], 1: weights[1]}

In [5]:
#Helper Function: Return the img paths and classes in seperate lists given a txt file from the LABELS folder
def getDirsAndClasses(root: str, file: str) -> Tuple[List[str], List[int]]:
    imageDirs = []
    classes = []
    line = ""
    with open(root + file, "r") as f:
        for line in tqdm(f):
            imageDir, clazz = line.split()
            imageDirs.append(imageDir)
            classes.append(int(clazz))
    return imageDirs, classes

In [6]:
#Helper Function: Create a Keras model
def makeModel(inputShape: (224, 224, 3)) -> keras.Model:
    inputs = keras.Input(shape=inputShape, name="Input")
    
    x = layers.Conv2D(filters=16, kernel_size=11, strides=4, activation="relu")(inputs)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)
    x = layers.Conv2D(filters=20, kernel_size=5, strides=1, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)
    x = layers.Conv2D(filters=30, kernel_size=3, strides=1, activation="relu")(x)
    x = layers.MaxPooling2D(pool_size=3, strides=2)(x)
    x = layers.Dense(units=48, activation="relu")(x)
    x = layers.Flatten()(x)
    outputs = layers.Dense(units=1, activation="sigmoid")(x)
    
    model = keras.Model(inputs=inputs, outputs=outputs, name="mAlexNet")
    return model

In [7]:
#Get the Train Dataset using split from the LABELS folder
root = os.getcwd() + "\\Data\\CNR-EXT-150x150"
imageDirs, classes = getDirsAndClasses(root, "\\LABELS\\train.txt")
root = root + "\\PATCHES\\"
train = pd.DataFrame([
            {
                "image": root + filename,
                "class": "free" if clazz == 0 else "busy"
            }
            for filename, clazz in tqdm(zip(imageDirs, classes))
    ])
#Now Get Test
root = os.getcwd() + "\\Data\\CNR-EXT-150x150"
imageDirs, classes = getDirsAndClasses(root, "\\LABELS\\test.txt")
root = root + "\\PATCHES\\"
test = pd.DataFrame([
            {
                "image": root + filename,
                "class": "free" if clazz == 0 else "busy"
            }
            for filename, clazz in tqdm(zip(imageDirs, classes))
    ])

94493it [00:00, 687026.21it/s]
94493it [00:00, 655644.26it/s]
31825it [00:00, 587844.99it/s]
31825it [00:00, 1413288.92it/s]


In [8]:
#Declare data generators and preprocessing
train_datagen = ImageDataGenerator(
    #Augment data with random flips, normalize each sample's input
    vertical_flip = True,
    horizontal_flip = True,
    rescale = 1.0 / 255.0,
    samplewise_std_normalization = True
)
train_generator = train_datagen.flow_from_dataframe(
    directory = None, #none since the df has absolute paths
    dataframe = train,
    x_col = "image",
    y_col = "class",
    validate_filenames = False, #faster for huge datasets
    target_size = (224, 224),
    color_mode = "rgb",
    batch_size = 128,
    class_mode = "binary",
    shuffle = True
)

test_datagen = ImageDataGenerator(
    samplewise_std_normalization = True
)
test_generator = train_datagen.flow_from_dataframe(
    directory = None,
    dataframe = test,
    x_col = "image",
    y_col = "class",
    validate_filenames = False,
    target_size = (224, 224),
    color_mode = "rgb",
    batch_size = 128,
    class_mode = "binary",
    shuffle = True
)

C:\Users\music\anaconda3\envs\csc790\lib\site-packages\keras_preprocessing\image\image_data_generator.py:356: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


Found 94493 non-validated image filenames belonging to 2 classes.
Found 31825 non-validated image filenames belonging to 2 classes.


C:\Users\music\anaconda3\envs\csc790\lib\site-packages\keras_preprocessing\image\image_data_generator.py:356: UserWarning: This ImageDataGenerator specifies `samplewise_std_normalization`, which overrides setting of `samplewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [9]:
#Declare Callbacks: stop training if accuracy doesn't rise 1% within 3 epochs
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor = "accuracy",
        min_delta = 0.01,
        patience = 3,
        verbose = 1
    )
]

In [10]:
#Extract Class Weights
classes = list(train["class"])
weights_dict = getClassWeightsFromLabels(classes)
print(weights_dict)

{0: 0.992240003360215, 1: 1.0078823303539048}


In [11]:
#Build Model
Model = makeModel((224, 224, 3))
opt = tf.optimizers.Adam()
Model.compile(
    optimizer = opt,
    loss = keras.losses.BinaryCrossentropy(from_logits = True),
    metrics = ["accuracy"]
)

In [12]:
Model.summary()

Model: "mAlexNet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Input (InputLayer)           [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv2d (Conv2D)              (None, 54, 54, 16)        5824      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 26, 26, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 20)        8020      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 10, 10, 20)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 8, 8, 30)          5430      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 3, 3, 30)          0  

In [13]:
#Fit data
Model.fit(
    train_generator,
    callbacks = callbacks,
    epochs = 100,
    class_weight = weights_dict,
    max_queue_size = 1000,
    workers = os.cpu_count()
)

Epoch 1/100
739/739 [==============================] - 54s 73ms/step - loss: 0.5501 - accuracy: 0.8954
Epoch 2/100
739/739 [==============================] - 55s 74ms/step - loss: 0.5360 - accuracy: 0.9286
Epoch 3/100
739/739 [==============================] - 55s 74ms/step - loss: 0.5319 - accuracy: 0.9378
Epoch 4/100
739/739 [==============================] - 55s 74ms/step - loss: 0.5286 - accuracy: 0.9457
Epoch 5/100
739/739 [==============================] - 55s 75ms/step - loss: 0.5270 - accuracy: 0.9485
Epoch 6/100
739/739 [==============================] - 55s 75ms/step - loss: 0.5294 - accuracy: 0.9434
Epoch 7/100
739/739 [==============================] - 56s 75ms/step - loss: 0.5277 - accuracy: 0.9469
Epoch 00007: early stopping


In [14]:
#Test accuracy
Model.evaluate(
    test_generator,
    max_queue_size = 1000,
    workers = os.cpu_count()
)

249/249 [==============================] - 40s 159ms/step - loss: 0.5538 - accuracy: 0.9485s - loss: 0.5 - ETA: 0s - loss: 0.5539 - accura


[0.5538106560707092, 0.9485310316085815]

In [15]:
#Save the model
Model.save("Models/mAlexNet")

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: Models/mAlexNet\assets
